In [1]:
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
import albumentations as A
import tensorflow as tf

In [2]:
dataset = Dataset.carabid
dataset_name = str(dataset).split(".")[1]
raw_train, raw_val = prep_dataset(dataset, 8)

Found 63364 files belonging to 291 classes.
Using 53860 files for training.
Found 63364 files belonging to 291 classes.
Using 9504 files for validation.


Tests the classification accuracy of extractors trained with standard (un-augmented), blurred and greyed images on each of the three image types:

In [3]:
standard_model_path = f"../model-saves/extractors/{dataset_name}/{dataset_name.upper()}-EXTRACTOR/extractor/savefile.hdf5"
standard_model = keras.models.load_model(standard_model_path)

gray_model_path = f"../model-saves/extractors/{dataset_name}/{dataset_name.upper()}-GRAY-EXTRACTOR/extractor/savefile.hdf5"
gray_model = keras.models.load_model(gray_model_path)

blur_model_path = f"../model-saves/extractors/{dataset_name}/{dataset_name.upper()}-BLUR-EXTRACTOR/extractor/savefile.hdf5"
blur_model = keras.models.load_model(blur_model_path)

gray_aug = A.Compose([A.ToGray(p=1.0)])
blur_aug = A.Compose([A.Blur(p=1.0)])

def apply_aug_gray(images):
    aug_imgs = []
    for img in images:
        aug_imgs.append(gray_aug(image=img)["image"])
    return np.array(aug_imgs)

def apply_aug_blur(images):
    aug_imgs = []
    for img in images:
        aug_imgs.append(blur_aug(image=img)["image"])
    return np.array(aug_imgs)

def process_data_gray(images, labels):
    aug_imgs = tf.numpy_function(apply_aug_gray, [images], tf.float32)
    return aug_imgs, labels

def process_data_blur(images, labels):
    aug_imgs = tf.numpy_function(apply_aug_blur, [images], tf.float32)
    return aug_imgs, labels

standard_val = raw_val
gray_val = raw_val.map(process_data_gray)
blur_val = raw_val.map(process_data_blur)

models = {"standard": standard_model, "gray": gray_model, "blur": blur_model}
datasets = {"standard": standard_val, "gray": gray_val, "blur": blur_val}
results = {}

for m_name, m in models.items():
    for d_name, d in datasets.items():
        print(f"Testing {d_name} dataset on {m_name} model:")
        results[(m_name, d_name)] = m.evaluate(d)

print(results)

Testing standard dataset on standard model:
1188/1188 [==============================] - 40s 29ms/step - loss: 0.3929 - accuracy: 0.8961
Testing gray dataset on standard model:
1188/1188 [==============================] - 42s 35ms/step - loss: 5.9422 - accuracy: 0.1652
Testing blur dataset on standard model:
1188/1188 [==============================] - 46s 39ms/step - loss: 6.0038 - accuracy: 0.3431
Testing standard dataset on gray model:
1188/1188 [==============================] - 36s 30ms/step - loss: 0.6064 - accuracy: 0.8342
Testing gray dataset on gray model:
1188/1188 [==============================] - 42s 35ms/step - loss: 0.6121 - accuracy: 0.8333
Testing blur dataset on gray model:
1188/1188 [==============================] - 46s 39ms/step - loss: 6.1340 - accuracy: 0.2635
Testing standard dataset on blur model:
1188/1188 [==============================] - 36s 29ms/step - loss: 0.4149 - accuracy: 0.8858
Testing gray dataset on blur model:
1188/1188 [==========================